In [1]:
import utils2
sales_util = utils2.SalesUtils('')

import pandas as pd 
import numpy as np
import time
import gc
import os
import pickle

from itertools import product

from collections import Counter
import re
from operator import itemgetter
from sklearn.preprocessing import LabelEncoder

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
%%time
input_path = '../data/'
d_parser = lambda x: pd.datetime.strptime(x,'%d.%m.%Y')
df_sales           = pd.read_csv(os.path.join(input_path, 'sales_train.csv'), parse_dates =["date"],date_parser=d_parser)
df_items           = pd.read_csv(os.path.join(input_path, 'items.csv'))
df_item_categories = pd.read_csv(os.path.join(input_path, 'item_categories.csv'))
df_shops           = pd.read_csv(os.path.join(input_path, 'shops.csv'))
df_test            = pd.read_csv(os.path.join(input_path, 'test.csv'))

Wall time: 1min 40s


# Train Set Pipeine-1
Cleaning, Imputation, Outliers, Merging, Features etc

In [3]:
%%time
############ DELETING OUTLIERS 
# deleting the quantities which are greater than 1001
df_sales = df_sales[df_sales['item_cnt_day']<=1001]
#any item_cnt_day which is less than 0 should be 0
df_sales.loc[df_sales.item_cnt_day < 1,'item_cnt_day'] = 0 

# values more than 55k could be outliers, so deleting all above 55k
df_sales = df_sales[df_sales['item_price']<= 55000]

#item price, should obviously not be less than 0 ... either it should be deleted or imputed.
df_sales = df_sales[df_sales['item_price'] > 0]

############ ADDING DATE ATTRIBUTES
# Adding the date time attributes (like week day, month number, etc.)
df_sales = sales_util.add_date_attributes(df_sales)

############ REPLACING DUPLICATE SHOPS FROM SALES
# based on the above, duplicating as follows
df_sales['shop_id'].replace({0: 57, 1: 58, 11: 10, 40: 39}, inplace=True)

############ REMOVING OUTDATED SHOPS FROM SALES
outdated_shops = [0, 1, 8, 11, 13, 17, 23, 29, 30, 32, 33, 40, 43, 54]
df_sales = df_sales[df_sales['shop_id'].isin(outdated_shops)==False]

############ MERGING WITH THE SALES AND SHOPS AND ITEMS/ITEM_CATEGORIES
df_sales = sales_util.merge_items_sales_n_shops(df_sales)

############ CREATING THE MONTHLY REVENUE
df_sales['revenue'] = df_sales['item_cnt_day'] * df_sales['item_price']

Wall time: 11.1 s


# Test Set Pipeine-1
Cleaning, Imputation, Outliers, Merging, Features etc

In [6]:
%%time
#1. test data frame, adding date_block_num and month
df_test['date_block_num'] = 34
df_test['month'] = 11

############ REPLACING DUPLICATE SHOPS FROM SALES
# based on the above, duplicating as follows
df_sales['shop_id'].replace({0: 57, 1: 58, 11: 10, 40: 39}, inplace=True)

#2. merging, just like the sales
df_test = sales_util.merge_items_sales_n_shops(df_test)

############ REPLACING DUPLICATE CATEGORY
# duplicate category id
df_test['item_category_id'].replace({8: 80, 27: 74, 75: 76}, inplace=True)

Wall time: 1.2 s


# Train Set Pipeine-2
Advanced feature generation, monthly grouping, merge with the Test set

In [ ]:
df_shops.shop_name.unique()

In [ ]:
df_shops.groupby(['shop_cat']).count()

In [ ]:
df_shops['city'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[0])
df_shops['shop_cat'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[1]).astype(str)
df_shops.loc[df_shops.city == '!Якутск', 'city'] = 'Якутск'
shop_category = ['ТК','ТРК','ТРЦ','ТЦ']
df_shops.shop_cat = df_shops.shop_cat.apply(lambda x: x if (x in shop_category) else 'etc')

In [ ]:
df_shops['shop_cat'].unique()

In [ ]:
df_shops.groupby(['city']).sum()

In [ ]:
len(df_shops[df_shops.shop_cat=='ТЦ'])

In [ ]:
df_shops.shop_cat.sum()

In [ ]:
df_items.item_name.unique()

In [ ]:
df_items['name_1'], df_items['name_2'] = df_items['item_name'].str.split('[', 1).str
df_items['name_1'], df_items['name_3'] = df_items['item_name'].str.split('(', 1).str


df_items['name_2'] = df_items['name_2'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
df_items['name_3'] = df_items['name_3'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ').str.lower()
df_items = df_items.fillna('0')

In [ ]:
df_items[['item_name','name_1','name_2','name_3']]

In [ ]:
#df1 = Counter(' '.join(df_items['name_2'].values.tolist()).split(' ')).items()
df1 = Counter(' '.join(df_items['name_2'].values).split(' ')).items()
df1 = sorted(df1, key=itemgetter(1))
df1 = pd.DataFrame(df1, columns=['feature', 'count'])
df1.fillna(0)
df1 = df1[(df1['feature'].str.len() > 1) & (df1['count'] > 200)]

df2 = Counter(' '.join(df_items['name_3'].values).split(' ')).items()
df2 = sorted(df2, key=itemgetter(1))
df2 = pd.DataFrame(df2, columns=['feature', 'count'])
df2.fillna(0)
df2 = df2[(df2['feature'].str.len() > 1) & (df2['count'] > 200)]

item_feature_set = pd.concat([df1,df2])
item_feature_set = item_feature_set.drop_duplicates(subset=['feature']).reset_index(drop=True)

def name_correction(x):
    x = x.lower()
    x = x.partition('[')[0]
    x = x.partition('(')[0]
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)
    re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x)
    x = x.strip()
    return x

df_items['item_name'] = df_items['item_name'] .apply(lambda x: name_correction(x))
df_items.name_2 = df_items.name_2.apply(lambda x: x[:-1] if x!='0' else '0')

In [ ]:
df_items['type'] = df_items.name_2.apply(lambda x: x[0:8] if x.split(' ')[0] == 'xbox' else x.split(' ')[0])
df_items.loc[(df_items.type == 'x360') | (df_items.type == 'xbox360'), 'type'] = 'xbox 360'
df_items.loc[df_items.type == '', 'type'] = 'mac'
df_items.type = df_items.type.apply(lambda x: x.replace(' ',''))

df_items.loc[(df_items.type == 'pc') | (df_items.type == 'pс') | (df_items.type == 'рс'), 'type'] = 'pc'
df_items.loc[(df_items.type == 'рs3'), 'type'] = 'ps3'

In [ ]:
df_items[df_items.type=='psp']

In [ ]:
group_sum = df_items.groupby('type', as_index=False).sum()

#the sum of all item_category_ids under a particular type shoudl be greater than 200
to_del_types = group_sum.loc[group_sum.item_category_id < 200].type.tolist() 
group_sum.loc[group_sum.item_category_id < 200] 

In [ ]:
to_del_types

In [ ]:
df_items.name_2 = df_items.type.apply(lambda x: 'etc' if x in to_del_types else x)
df_items = df_items.drop(['type'], axis=1)
df_items.groupby(['name_2']).sum()

In [ ]:
df_items.head()

In [ ]:
df_items['name_2'] = LabelEncoder().fit_transform(df_items['name_2'])
df_items['name_3'] = LabelEncoder().fit_transform(df_items['name_3'])
df_items.drop(['item_name', 'name_1'], axis=1, inplace=True)
df_items.head()

In [ ]:
t = Counter(s).items()

In [ ]:
t